In [1]:
import numpy as np
import random
from collections import defaultdict

In [2]:
class IndependentCascade():
    def __init__(self, delta=1e-5, maxIter=500):
        self.preds = defaultdict(dict)
        self.delta = delta
        self.maxIter = maxIter
        self.succs = defaultdict(dict)
    def createGraph(self,episodes):
        for ep in episodes:
            succs = [ep[ep[:,1] > ep[i,1]][:,0] for i in xrange(len(ep))]
            for i,succ in enumerate(succs):
                for s in succ:
                    r = np.random() 
                    self.preds[int(s)][ep[i,0]] = r
                    self.succs[ep[i,0]][int(s)] = r
    def fit(self, episodes):
        self.episodes = episodes
        stop = False
        it = 0
        while not stop:
            p = {}
            
            for d,episode in enumerate(episodes):
                times = np.unique(episode[:,1])
                users = episode[:,0]
        
                #Calcul de PDtu
                p[d] = np.zeros((len(times),len(users)))
                p[d][0,episode[:,1] == times[0]] = 1
                
                for t in xrange(1,len(times)):
                    for u,user in enumerate(users):
                        hasPreds = False
                        pdtu = 1.
                        #for v in episode[episode[:,1] == times[t-1]][:,0]:
                        for v in episode[episode[:,1] < times[t]][:,0]:
                            if (v in self.preds[user]):
                                hasPreds = True
                                pdtu = pdtu * (1 - self.preds[user][v])
                        p[d][t,u] = 1-pdtu if hasPreds else 1
    
            #Maximisation de la vraisemblance    
            dplus  = np.zeros((len(self.preds), len(self.preds)))
            ##dminus = np.ones((len(self.preds), len(self.preds))) * len(episodes) : U n'apparait pas dans tous les ep
            dminus = np.zeros((len(self.preds), len(self.preds)))
            theta  = np.zeros((len(self.preds), len(self.preds)))
            for d,episode in enumerate(episodes):
                times = list(np.unique(episode[:,1]))
                users = episode[:,0]
                for u,uId in enumerate(users):
                    ##
                    dminus[uId,:] = dminus[uId,:] + 1
                    for v,vId in enumerate(users):
                        #if (episode[v,1] <= episode[u,1] + 1):
                        #    dminus[uId, vId] = dminus[uId, vId] - 1
                        dminus[uId, vId] = dminus[uId, vId] - 1
                        #if (episode[v,1] == episode[u,1] + 1):
                        if (episode[v,1] > episode[u,1]):
                            dplus[uId, vId] = dplus[uId, vId] + 1
                            tv = times.index(episode[v,1])
                            theta[uId, vId] = theta[uId, vId] + (self.preds[vId][uId] / p[d][tv, v])
            theta = theta / (dplus + dminus)
            for u in self.preds:
                for v in self.preds[u]:
                    self.preds[u][v] = theta[v,u]
                    self.succs[v][u] = theta[v,u]
                    
            #Calcul de la vraisemblance / MAP
            logL = it + 1
            print it, self.score(episodes)
            
            if (it != 0) and ((it == self.maxIter) or (logL - prevLogL < self.delta)):
                stop = True
            it = it+1
            prevLogL = logL
    def inference(self, s0):
        #infected = np.zeros((len(self.succs)+1), bool)
        infected = defaultdict(bool)
        s = []
        s.append(s0)
        t = 1
        stop = False
        while s[t-1] != []:
            s.append([])
            for i in s[t-1]:
                for j in self.succs[i].keys():
                    if (not infected[j]) and (random.random() < self.succs[i][j]):
                        infected[j] = True
                        s[t].append(j)
            t = t + 1
        return s, infected
    def predict(self, s0, nIter=10000):
        #infected = np.zeros((len(self.succs)+1), int)
        suminfected = defaultdict(float)
        for i in xrange(nIter):
            _, infected = self.inference(s0)
            for j in infected.keys():
                suminfected[j] = suminfected[j] + infected[j]
        for j in xrange(len(suminfected)):
            suminfected[j] = suminfected[j] / nIter
        return suminfected
    def score(self, episodes, nIter=10000):
        score = 0
        for episode in episodes:
            times = np.unique(episode[:,1])
            users = episode[:,0]
            sources = users[[episode[:,1] == times[0]]]
            pred = self.predict(sources, nIter)
            rank = np.array(pred.keys())[(-np.array(pred.values())).argsort()]
            scoreEp = 0
            count = 0.0
            for i,u in enumerate(rank):
                if u in users:
                    count += 1.0
                    scoreEp += count / (i+1)
            score += scoreEp / len(users)
        score /= len(episodes)
        return score        
                

In [3]:
def loadEpisodes(datafile="./train"):
    f = open(datafile)
    episodes = []
    for episode in f.readlines():
        ep = episode[:-3]
        ep = np.array([ept.split(":") for ept in ep.split(";")], float)
        ep = np.array(ep, int)
        episodes.append(ep[ep[:,1].argsort()])
    return np.array(episodes)

In [4]:
episodes = loadEpisodes()

In [5]:
ic = IndependentCascade(maxIter=1000)
ic.createGraph(episodes)

In [15]:
import pickle
pickle.dump(dict(ic.preds), open("/Vrac/3000693/FDMS/preds.pkl", "w"))
pickle.dump(dict(ic.succs), open("/Vrac/3000693/FDMS/succs.pkl", "w"))
pickle.load(open("/Vrac/3000693/FDMS/preds.pkl",'r'))

{0: {1: 0.03763760917597701,
  2: 0.020431650848578637,
  3: 0.043299094090839776,
  4: 0.02189203348110662,
  5: 0.008365670063014119,
  6: 0.03856239087088459,
  7: 0.016142169819653985,
  8: 0.02254206727905885,
  9: 0.03713783056291366,
  10: 0.0329161807621964,
  11: 0.040732176945260885,
  12: 0.025975294208793055,
  13: 0.04614612318707547,
  14: 0.046876561528888507,
  15: 0.020431911858624013,
  16: 0.007612194635179048,
  17: 0.04470280846928551,
  18: 0.00700158269607557,
  19: 0.011377596985861427,
  20: 0.02401220336321704,
  21: 0.03477848269982013,
  22: 0.017890838901318706,
  23: 0.02791507838556499,
  24: 0.03733314373860724,
  25: 0.010733240294180274,
  26: 0.03963185733818379,
  27: 0.030361509140193955,
  28: 0.04788154933562573,
  29: 0.03018527608607789,
  30: 0.028818973563678286,
  31: 0.019116468199176413,
  32: 0.010979244006615407,
  33: 0.004041923978832335,
  34: 0.01926872565594509,
  35: 0.01768671256508303,
  36: 0.047887286822054384,
  37: 0.015646305

In [5]:
ic = IndependentCascade(maxIter=1000)
ic.createGraph(episodes)
ic.fit(episodes)

0

KeyboardInterrupt: 